## Первичная обработка текста: токенизация

Прежде, чем находить и исправлять опечатки и ошибки, необходимо разбить текст на слова, знаки препинания и прочие языковые единицы. Этот процесс сегментации текста называется токенизации, а полученные единицы (сегменты) это токены.

С первого взгляда может показаться, что достаточно просто разбить текст по пробелам и мы получим все токены. Но останутся знаки препинания. Когда и они будут учтены — окажется, например, что точка служит не только как конец предложения, но и для написания дат (21.06.2024). Кроме того, есть ещё имена собственные (New York), аббревиатуры (НГУ, ЛШ), составные слова (кресло-кровать), неразрывные неизменяемые словосочетания (и_так_далее; таким_образом), интернет-адреса (http://yandex.ru) и т. д. Короче говоря, задача на самом деле не тривиальная, но мы постараемся её решить.

## Способы токенизации
Мы рассмотрим 3 способа: составим простой алгоритм на python, воспользуемся библиотекой и попробуем использовать регулярные выражения
### Простой алгоритм на python
Давайте начнём с простого, разобём текст по пробелам. Для этого на python существует метод split(), который разбивает строку (str) по выбранным символам. Рассмотрим пример:

In [ ]:
sentence = 'Все люди как пипл, а мы ФИПЛ'
sentence.split()


['Все', 'люди', 'как', 'пипл,', 'а', 'мы', 'ФИПЛ']

Несмотря на то, что аргумент метода split() пустой, о умолчанию он разбивает строку на пробелы. Если сделать то же самое, но указать в качестве аргумента пробел, мы получим тот же результат:

In [ ]:
sentence.split(' ')  # В кавычках пробел


['Все', 'люди', 'как', 'пипл,', 'а', 'мы', 'ФИПЛ']

Метод split() возвращает список (list) с элементами строки. Несложно заметить, что слово "пипл," содержит запятую. Чтобы избавиться от неё, необходимо применить split(',') к каждому элементу списка. Для этого воспользуемся циклом for, который будет "пробегать" по каждому элементу списка и что-то с ним делать. Рассмотрим на примере:

In [ ]:
for element in sentence.split(' '):  # для каждого element в списке
  print(element)  # выводим на экран сам element


Все
люди
как
пипл,
а
мы
ФИПЛ


Воспользуйся полученными знаниями и добавь внутрь цикла метод split(), указав в качестве аргумента запятую:

In [ ]:
for element in sentence.split(' '):
  # Напиши внутри print метод split() для element
  print()


In [ ]:
for element in sentence.split(' '):
  # Напиши внутри print метод split() для element
  print(element.split(','))

['Все']
['люди']
['как']
['пипл', '']
['а']
['мы']
['ФИПЛ']


Надеюсь, что у тебя всё получилось! Если бы мы учитывали все тонкости, то у нас ушло бы на это очень много времени. Поэтому рассмотрим другие решения нашей задачи.

### Профессиональное решение
Воспользуемся библиотекой spaCy, для этого её нужно импортировать, воспользуемся кодом ниже:

In [ ]:
import spacy

Google colab это популярная платформа для работы с нейросетями, поэтому нам не придётся устанавливать библиотеку. И да, spaCy использует нейросеть для токенизации и другой обработки текста. Чтобы воспользоваться spaCy нужно загрузить малую обученную модель, не будем углубляться в детали и просто воспользуемся кодом ниже:

In [ ]:
nlp = spacy.load("en_core_web_sm")

Теперь мы просто используем nlp как функцию для обработки строки

In [ ]:
description = """
In computing, a pipeline, also known as a data pipeline,
is a set of data processing elements connected in series,
where the output of one element is the input of the next one.
The elements of a pipeline are often executed in parallel
or in time-sliced fashion.
— Wikipedia
"""

nlp(description)


In computing, a pipeline, also known as a data pipeline, 
is a set of data processing elements connected in series, 
where the output of one element is the input of the next one. 
The elements of a pipeline are often executed in parallel 
or in time-sliced fashion. 
— Wikipedia

Выглядит не так, как раньше, ведь это не привычный нам список (list). Давайте преобразуем его с помощью функции list()

In [ ]:
list(nlp(description))

[,
 In,
 computing,
 ,,
 a,
 pipeline,
 ,,
 also,
 known,
 as,
 a,
 data,
 pipeline,
 ,,
 ,
 is,
 a,
 set,
 of,
 data,
 processing,
 elements,
 connected,
 in,
 series,
 ,,
 ,
 where,
 the,
 output,
 of,
 one,
 element,
 is,
 the,
 input,
 of,
 the,
 next,
 one,
 .,
 ,
 The,
 elements,
 of,
 a,
 pipeline,
 are,
 often,
 executed,
 in,
 parallel,
 ,
 or,
 in,
 time,
 -,
 sliced,
 fashion,
 .,
 ,
 —,
 Wikipedia,
 ]

Вот это уже похоже на правду! Давайте ближе к делу. Мы хотим получить из текста только слова, которые легко распознать как слова с опечаткой. То есть мы пометим все знаки препинания, имена собственные и так далее, чтобы потом мы могли их отсеять.

Для этого мы воспользуемся словарём. Словарь это набор пар ключ-значени. Рассмотрим на примере:

In [ ]:
ages_dict = {
    'Haisenber': '52',
    'Jesse': '23',
}

ages_dict

{'Haisenber': '52', 'Jesse': '23'}

В качестве ключа нашего словаря будет токен, а в качестве значения True или False. Если слово НЕ подходит для рассмотрения на предмет опечаток, то True, иначе False. Рассмотрим на примере:

In [ ]:
tokens_dict = {
    'Walter White': True,  # Имя собственное не рассматриваем
    'cat': False,
    'NSU': True,  # Аббревиатуры не рассматриваем
    'bread': False
}

tokens_dict

{'Walter White': False, 'cat': True, 'NSU': False, 'bread': True}

Теперь нам нужно сделать словарь из тех токенов, которые мы получили с помощью spaCy. Здесь важно кое-что прояснить: токены spaCy это не просто текст, но также и другая лингвистическая информация, например часть речи. Рассмотрим на примере из документации spaCy:

In [ ]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text, token.pos, token.pos_)

Apple 96 PROPN
is 87 AUX
looking 100 VERB
at 85 ADP
buying 100 VERB
U.K. 96 PROPN
startup 92 NOUN
for 85 ADP
$ 99 SYM
1 93 NUM
billion 93 NUM


У токена есть разные атрибуты, то есть переменные, которые принадлежат этому объекту. Например, атрибут `text` это токен в привычном нам формате str. Атрибуты `pos` и `pos_` это часть речи (Part Of Speech, pos) в разной записи — в формате числа и строки.

Таких атрибутов у токена довольно много, но нас будут интересовать только текст и часть речи: текст будем использовать в качестве ключа словаря, а значение (True/False) определим на основе части речи. То есть какие-то части речи мы будем рассматривать для исправления опечаток, а какие-то нет. Рассмотрим на примере:

In [ ]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    key = token.text  # Текст токена будет ключем в словаре
    value = token.pos_ != 'VERB'  # Если часть речи токена это VERB (глагол),
                                  # То тогда значение в словаре False,
                                  # иначе True

    print(key, value)

Apple False
is False
looking True
at False
buying True
U.K. False
startup False
for False
$ False
1 False
billion False


В данном примере мы используем цикл, в котором пробегаемся по всем токенам текста. В качестве ключа мы используем текст токена

```key = token.text   # Переменная key содержит текст токена```

В качестве значения мы хотим получить True/False в зависимости от части речи:

```value = token.pos_ != 'VERB'```

Символ `!=` проверяет неравенство значений слева и справа от знака. Если часть речи токена это VERB, то мы получим False и переменная value будет содержать False, в ином случае True.

Вот мы и составили пары ключ-значение на основе текста. Теперь осталось отсеять все ненужные нам части речи. Мы уже подготовили список не подходящих частей речи, запусти код ниже, чтобы иницилизоровать переменную banned_pos

In [ ]:
banned_pos = ['PROPN', 'SYM', 'PART', 'CCONJ', 'ADP', 'PUNCT']

Чтобы проверить, содержится ли часть речи в списке banned_pos, нужно написать следующее выражение:

In [ ]:
'VERB' in banned_pos

False

Теперь давайте решать нашу задачу. Для начала напиши код, который будет выводить на экран слово и False/True в зависимости от того, находится ли оно в banned_pos

In [ ]:
text = 'Jesse, we have to cook'

# Напиши свой код ниже


In [ ]:
text = 'Jesse, we have to cook'

# Напиши свой код ниже
doc = nlp(text)
for token in doc:
  key = token.text
  value = token.pos_ in banned_pos
  print(key, value)

Jesse True
, True
we False
have False
to True
cook False


Отлично, теперь сделаем из этого словарь. Просто добавь перед циклом for переменную и иницилизируй словарь вот так:

```tokens_dict = dict()```

Затем вместо того, чтобы напечатать слово и значение True/False сделаем как пару ключ/значение

```
for token in doc:
  key =  # здесь текст токена
  value =  # здесь True, если часть речи в banned_pos
           # и False, если часть речи НЕ в banned_pos
  
  tokens_dict[key] = value  # создаём пару ключ-значение
```

Напиши код ниже и выведи словарь на экран с помощью print(). Заметь, что в цикле появилась дополнительная проверка с комментарием "Исключим перенос строки". Не удаляй её и вставь в последующие циклы.

In [ ]:
text = 'Jesse, we have to cook'

# Иницилизируем части речи, которые мы не будем рассматривать
banned_pos = ['PROPN', 'SYM', 'PART', 'CCONJ', 'ADP', 'PUNCT']

# Иницилизируй словарь перед циклом


for token in doc:
  key =  # Присвой переменной key значение текста токена
  value =   # Присвой переменной value True/False в зависимости от части речи

  # Исключим перенос строки, потому что его часть речи неизвестна
  # и если мы добавим её в banned_pos, то пропустим все опечатки,
  # потому что их часть речи также неизвестна
  if key == '\n':
    value = True

  # Создай пару ключ-значение


print()  # Напиши переменную словаря внутрь функции print


SyntaxError: invalid syntax (<ipython-input-75-74dee08bf0e2>, line 7)

In [ ]:
text = 'Jesse, we have to cook'

# Иницилизируем части речи, которые мы не будем рассматривать
banned_pos = ['PROPN', 'SYM', 'PART', 'CCONJ', 'ADP', 'PUNCT']

# Иницилизируй словарь перед циклом
tokens_dict = dict()

for token in doc:
  key = token.text
  value = token.pos_ in banned_pos

  # Исключим перенос строки, потому что его часть речи неизвестна
  # и если мы добавим её в banned_pos, то пропустим все опечатки,
  # потому что их часть речи также неизвестна
  if key == '\n':
    value = True

  # Создай пару ключ-значение
  tokens_dict[key] = value

print(tokens_dict)  # Напиши переменную словаря внутрь функции print


{'Jesse': True, ',': True, 'we': False, 'have': False, 'to': True, 'cook': False}


Отлично, теперь осталось оформить всё это в функции. Функция работает очень просто: мы даём ей какое-то название, определяем, будет ли она что-то принимать в качестве аргумента и затем пишем внутри то, что она будет делать. Вот так:

In [ ]:
def print_tokens(text):  # Def это функция, print_string это название,
                           # String это аргумент
  print(text.split())

text = 'Jesse, we have to cook'  # Напишем какую-нибудь строку
print_tokens(text)  # Вызовем функцию и передадим ей эту строку

['Jesse,', 'we', 'have', 'to', 'cook']


Теперь финальное задание. У нас есть функция get_tokens_dictionary, её аргумент это какая-то строка. Нам нужно взять тот код, который на основе строки создаёт словарь и вставить его внутрь функции.

И самое главное, создание словаря и его заполнение должно быть внутри функции.

In [ ]:
def get_tokens_dict(text):
  # Иницилизируем части речи, которые мы не будем рассматривать
  banned_pos = ['PROPN', 'SYM', 'PART', 'CCONJ', 'ADP', 'PUNCT']

  # Обработай text с помощью nlp и сохрани обработанный текст в doc


  # Иницилизируй словарь


  # Заполни словарь токенами и True/False c помощью цикла for


  return tokens_dict

In [ ]:
def get_tokens_dict(text):
  # Иницилизируем части речи, которые мы не будем рассматривать
  banned_pos = ['PROPN', 'SYM', 'PART', 'CCONJ', 'ADP', 'PUNCT']

  # Обработай text с помощью nlp и сохрани обработанный текст в doc
  doc = nlp(text)

  # Иницилизируй словарь
  tokens_dict = dict()

  # Заполни словарь токенами и True/False c помощью цикла for
  for token in doc:
    key = token.text
    value = token.pos_ in banned_pos

    # Исключим перенос строки, потому что его часть речи неизвестна
    # и если мы добавим её в banned_pos, то пропустим все опечатки,
    # потому что их часть речи также неизвестна
    if key == '\n':
      value = True

    tokens_dict[key] = value

  return tokens_dict

In [ ]:
# Запусти этот код, чтобы проверить, работает ли функция

sample_text = '''
Levenshtein distnce, also knowm as edit distance, is a metric
fo measuring the difference between two sequencess.
It represents the minimum numbet of single-character editk
(insertions, deletons, or substititions) requirуd to change
one word or string ingo the other.
'''

get_tokens_dict(sample_text)

{'\n': True,
 'Levenshtein': True,
 'distnce': False,
 ',': True,
 'also': False,
 'knowm': False,
 'as': True,
 'edit': False,
 'distance': False,
 'is': False,
 'a': False,
 'metric': False,
 'fo': True,
 'measuring': False,
 'the': False,
 'difference': False,
 'between': True,
 'two': False,
 'sequencess': False,
 '.': True,
 'It': False,
 'represents': False,
 'minimum': False,
 'numbet': False,
 'of': True,
 'single': False,
 '-': True,
 'character': False,
 'editk': False,
 '(': True,
 'insertions': False,
 'deletons': False,
 'or': True,
 'substititions': False,
 ')': True,
 'requirуd': False,
 'to': True,
 'change': False,
 'one': False,
 'word': False,
 'string': False,
 'ingo': True,
 'other': False}

Надеюсь, что всё работает! Далее мы будем использовать эту функцию в приложении, поэтому не сохрани свой результат (ctrl + s)

### Алтернативный способ: регулярные выражения
Задача решена. Но какие есть другие способы токенизировать текст? Одним из неплохих решений можно считать использование регулярных выражений, давайте посмотрим что это такое

In [3]:
import re  # Импортируем библиотеку регулярных выражений

sentence = 'Все люди, как пипл, а мы ФИПЛ'
print(re.split(' |, ', sentence), 'regex в деле')
print(sentence.split(' |, '), 'встроенный split')

['Все', 'люди', 'как', 'пипл', 'а', 'мы', 'ФИПЛ'] regex в деле
['Все люди, как пипл, а мы ФИПЛ'] встроенный split


Как же это работает? Регулярные выражения позволяют использовать в качестве паттерна различные символы. Кажется, стало только непонятней. Рассмотрим на примере:

`re.split(' |, ', sentence)`

re.split использует в качестве аргумента для разбиения строки паттерн в кавычках `' |, '`, в котором указан символ `|`, он же "или". Этот символ позволяет выбирать по каким символам разбивать строку: по пробелу (указан слева от `|`) или по запятой и пробелу (указаны справа от `|`). Поэтому в итоге слово "пипл" не содержит запятой, ведь после него стояла запятая и пробел, которую учёл regex.

Но что, если мы хотим учесть точку и другие знаки препинания? Для этого воспользуемся в паттерне квадратными скобками: они позволяют нам учесть все паттерны, в которых содержится один из элементов, указанных внутри квадратных скобок. Рассмотрим на примере:

In [ ]:
sentence = 'Все люди, как пипл, а мы фипл'
re.findall('[фп]ипл', sentence)

['пипл', 'фипл']

Метод findall позволяет найти всё то, что указано в паттерне. Как видно, в паттерне мы указали `'[фп]ипл'`, то есть мы ищем либо фипл (ф из квадратных скобок), либо пипл (п из квадратных скобок).

Теперь попробуй самостоятельно дополнить паттерн `' |, '` другими знаками препинания, используя квадратные скобки на месте запятой

In [ ]:
sentence = 'Все люди, как пипл, а мы ФИПЛ'
re.split(' |, ', sentence)

Теперь, когда мы знаем что такое регулярные выражение, что такое паттерн и какие символы он может использовать — рассмотрим как с помощью него можно осуществить токенизацию текста:

In [ ]:
prayer = """
Ave, Maria, gratia plena; Dominus tecum; benedicta tu in mulieribus,
et benedictus fructus ventris tui, Iesus. Sancta Maria, Mater Dei,
ora pro nobis peccatoribus, nunc et in hora mortis nostrae. Amen.
"""

re.findall('\w+', prayer)

['Ave',
 'Maria',
 'gratia',
 'plena',
 'Dominus',
 'tecum',
 'benedicta',
 'tu',
 'in',
 'mulieribus',
 'et',
 'benedictus',
 'fructus',
 'ventris',
 'tui',
 'Iesus',
 'Sancta',
 'Maria',
 'Mater',
 'Dei',
 'ora',
 'pro',
 'nobis',
 'peccatoribus',
 'nunc',
 'et',
 'in',
 'hora',
 'mortis',
 'nostrae',
 'Amen']

С помощью findall мы ищем всё, что подходит под паттерн `\w+`, который состоит из двух элементов: `\w` и `+`. `\w` идентичен `'[a-zA-Z0-9_]'`, а `+` это 1 или более символов. То есть мы ищем 1 или более символов из набора всех латинских букв (строчных и прописных), цифр и нижнего подчёркивания.

Проверь это, записав re.findall с использованием идентичного паттерна вместо `\w`

In [ ]:
prayer = """
Ave, Maria, gratia plena; Dominus tecum; benedicta tu in mulieribus,
et benedictus fructus ventris tui, Iesus. Sancta Maria, Mater Dei,
ora pro nobis peccatoribus, nunc et in hora mortis nostrae. Amen.
"""

re.findall('', prayer)  # Напиши свой паттерн

In [4]:
prayer = """
Ave, Maria, gratia plena; Dominus tecum; benedicta tu in mulieribus,
et benedictus fructus ventris tui, Iesus. Sancta Maria, Mater Dei,
ora pro nobis peccatoribus, nunc et in hora mortis nostrae. Amen.
"""

re.findall('[a-zA-Z0-9_]+', prayer)  # Напиши свой паттерн

['Ave',
 'Maria',
 'gratia',
 'plena',
 'Dominus',
 'tecum',
 'benedicta',
 'tu',
 'in',
 'mulieribus',
 'et',
 'benedictus',
 'fructus',
 'ventris',
 'tui',
 'Iesus',
 'Sancta',
 'Maria',
 'Mater',
 'Dei',
 'ora',
 'pro',
 'nobis',
 'peccatoribus',
 'nunc',
 'et',
 'in',
 'hora',
 'mortis',
 'nostrae',
 'Amen']

И это только малая часть функционала регулярных выражений. Дома можете поиграться с регулярными выражениями в тренажёре по ссылке https://regexlearn.com/learn/regex101